In [ ]:
try:
    # settings colab:
    import google.colab
except ModuleNotFoundError:    
    # settings local:
    %run "../../../common/0_notebooks_base_setup.py"
    from checkpoint_pipeline import *

---

<img src='../../../common/logo_DH.png' align='left' width=35%/>


# <h1><center><ins>PIPELINE</ins></center></h1>
<h1><center>Checkpoint —solución— :</center></h1>
<img src="img/00_pipelines.jpg" alt="Drawing" style="width: 400px;"/>

<a id="tabla_contenidos"></a> 
## Tabla de Contenidos

### <a href='#section_objetivos'>0. Objetivos de la Notebook</a>


### <a href='#section_repaso'>1. Lo que tenés que hacer es...</a>
- #### <a href='#section_reg_vs_clas'>1.1 ...cargar el dataset y preparar los datos:</a>
- #### <a href='#1.2'>1.2 ...aplicar el Pipeline+GridSearchCV:</a>

<a id="section_objetivos"></a> 
## 0. Objetivos de la Notebook

<div id="caja11" style="float:left;width: 100%;">
  <div style="float:left;width: 9%;"><img src="../../../common/icons/haciendo_foco.png" style="align:left"/> </div>
  <br>
  <div style="float:left;width: 85%;">
      <label>Poner en práctica lo aprendido en la notebook de práctica guiada.</label>
  <div style="float:left;width: 85%;">
      <label>Es importante que antes de la clase resuelvan esta notebook ya que es fundamental que sepan utilizar las herramientas que vimos en la práctica guiada para después trabajar el caso práctico en la clase.</label>        
</div>    
</div>

<a href='#tabla_contenidos'>Volver a TOC</a>

<a id="section_repaso"></a> 
## 1. Lo que tenés que hacer es...

<a id="section_reg_vs_clas"></a>
### 1.1 ...cargar el dataset y preparar los datos:

Vamos a probar cómo armar `pipeline` combinado con `GridSearchCV`. Para ello vamos a volver a utilizar el dataset sobre la calidad de vinos que puede encontrarse en la página de Kaggle: https://www.kaggle.com/uciml/red-wine-quality-cortez-et-al-2009

El dataset está asociado a variantes rojas de un vino Portuguez. En la página de kaggle pueden encontrar la descripción de qué significan todas las variables. 

Importemos y exploremos un poco el dataset: 

In [ ]:
import pandas as pd

df = pd.read_csv('../Data/winequality-red.csv',sep=';')
df.head()

In [ ]:
df.info()

Binarizamos la calidad del vino (columna quality) tomando como criterio arbitrario, todos los vinos que tengan una calidad igual o mayor a 6 los vamos a considerar de buena calidad (les pondemos un 1 como etiqueta), mientras que los que sean inferiores a 6 los consideraremos de baja calidad (y los etiquetaremos con un 0):

In [ ]:
df['quality'].value_counts()

In [ ]:
df['quality_bin']=df['quality'].apply(lambda x: 1 if x>=6 else 0)
df['quality_bin'].value_counts(normalize=True)

Separar la `X` (todas las features menos **quality** ni **quality_bin**) y la `y` (**quality_bin**), y separar los datos en aquellos que se van a usar para el entrenamiento y los que servirán para testear el modelo.

In [ ]:
from sklearn.model_selection import train_test_split

# Separamos las variables independientes de la target
X=df.drop(columns=['quality','quality_bin'])
y=df['quality_bin']

# Dividimos los datos en el set de train y el de test: 
X_train, X_test, y_train, y_test=train_test_split(X,y,test_size=0.3, random_state=30, stratify=y)
display(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

<a href='#tabla_contenidos'>Volver a TOC</a>

<a id="1.2"></a>
### 1.2 ...aplicar el Pipeline+GridSearchCV:

Ahora que los datos están preparados, utilizando `Pipeline`+`GridSearchCV`, armar un proceso general en el que:

**a-** se seleccionen todas las features menos **residual sugar**

**b-** se normalicen los datos

**c-** se aplique una `LogistiRegression`


Con el `GridSearchCV`, el objetivo es explorar si el: en el primer paso **(a)** es mejor seleccionar o no hacer la selección; en el segundo paso **(b)** cuál es el mejor escalador si `StandardScaler` o `MinMaxScaler`, y por último en el paso **(c)** cuál es el valor óptimo del hiperparámetro C de la `LogisticRegression`. 

**(i)** Crear la clase para seleccionar las features:

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin

class FeatureSelection(BaseEstimator, TransformerMixin):
    
    def __init__(self,selected_features):
        self.selected_features=selected_features
    
    def fit(self,X,y=None):
        return self
    
    def transform(self, X, y=None):
        return X[self.selected_features]

**(ii)** Armar los pasos para el `Pipeline` e instancia el `Pipeline`

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
import numpy as np

In [ ]:
selector=FeatureSelection(selected_features=X.drop(columns='residual sugar').columns.values)

In [ ]:
pasos = [('feature_engineering',selector),
         ('preprocesamiento', MinMaxScaler()), 
         ('clasificador', LogisticRegression(solver='lbfgs'))]

pipe=Pipeline(pasos)

**(iii)** Generar la grilla y aplicar `GridSearchCV` con la `LogisticRegression`

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold

folds=StratifiedKFold(n_splits=5,shuffle=True, random_state=42)

In [ ]:
param_grid = [{'feature_engineering':[selector, None],
               'preprocesamiento':[StandardScaler(), MinMaxScaler()],
               'clasificador__C':np.linspace(0.1,1000,100)}] #list(np.linspace(0,1000,100)

grid=GridSearchCV(pipe, param_grid, cv=folds)

grid.fit(X_train,y_train)

**(iv)** Identificar cuál fue el mejor modelo, cuál fue su score de validación y evaluar su performance con los datos de test

In [ ]:
grid.best_estimator_

In [ ]:
grid.best_params_

In [ ]:
grid.best_score_

In [ ]:
grid.best_estimator_.score(X_test,y_test)

<a href='#tabla_contenidos'>Volver a TOC</a>